# Profitable App Profiles for App Store and Google Play Markets

I am completing this project under the assumption that I am working for a company that builds Android and iOS mobile apps, and makes those apps available on Google Play and the App Store. 

The company only builds apps that are free to download and install, and the main source of revenue consists of in-app adds. Therefore, revenue for a given app is mostly influenced by the number of users who see and engage with the ads. 
 

The goal of this project is to analyze data to understand what type of apps are likely to attract more users on Google Play and the App Store.

I will be analyzing a data set containing about 10,000 [Android apps](https://www.kaggle.com/lava18/google-play-store-apps) from Google Play (collected in Aug 2018) and 7,000 [iOS apps](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) from the App Store (collected in July 2017). 

In [3]:
from csv import reader

opened_apple_store_file = open('/Users/lisacondoluci/Documents/AppleStore.csv')
read_apple_store_file = reader(opened_apple_store_file)
ios_data = list(read_apple_store_file)
opened_apple_store_file.close()
ios_header = ios_data[0]
ios_data = ios_data[1:]

opened_google_play_store_file = open('/Users/lisacondoluci/Documents/googleplaystore.csv')
read_google_play_store_file = reader(opened_google_play_store_file)
android_data = list(read_google_play_store_file)
opened_google_play_store_file.close()
android_header = android_data[0]
android_data = android_data[1:]

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new empty line after each row
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
 
print(ios_header)
print('\n')
explore_data(ios_data, 0, 5)
print('\n')
print(android_header)
print('\n')
explore_data(android_data, 0, 5)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']




['App', 'Category', 'Rating', 'Reviews', 'Size', 'Instal

After briefly exploring the data, the next step is to clean the data. I will remove any apps that are not *free*, and are not directed toward an *English-speaking* audience. 

In [ ]:
print(android_data[10472])

This row in the Google Play Store data has an error since the rating is 19, and the rating can only go up to 5 on Google Play Store, so I will delete it.

In [ ]:
del android_data[10472]

Now I will check for duplicate entries in Google Play Store data. I'll determine which duplicate entry has the highest number of reviews, which would imply it is the most recent entry. Then, I will remove all other entries for a given app.

In [ ]:
duplicate_apps = []
unique_apps = []

for app in android_data:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])
print('\n')
print('Actual expected length:', len(android_data) - len(duplicate_apps))

Next, in order to remove all duplicate rows, I will create a data dictionary that includes the name of each app, as well as the highest number of reviews of all duplicate entries. When this is completed, I will check if the length of this data dictionary is the same as the actual expected length above, which it is.

In [ ]:
reviews_max = {}

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews

len(reviews_max)

The next step is to put all of the cleaned entries into a list of lists, called `android_clean`.

In [ ]:
android_clean = []
already_added = []

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)

According to the ASCII, the numbers corresponding to characters we commonly use in English are all in the range 0 to 127. Therefore, using the function `ord()` will help determine the numbers associated with each letter of a string (the app name). I will define a function `english(string)` that will input a string and determine if it is written in English. In order to prevent deleting apps incorrectly, I will only remove an app that has more than three characters outside of the ASCII range. I will test this function with 4 app names below.

In [ ]:
def english(string):
    non_english = 0
    for letter in string:
        if ord(letter) > 127:
            non_english += 1 
    if non_english > 3:
        return False
    else:
        return True

print(english('Instagram'))
print(english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english('Docs To Go™ Free Office Suite'))
print(english('Instachat 😜'))

I will now filter out any non-English apps from both data sets.

In [ ]:
english_ios_apps = []
for app in ios_data:
    name = app[1]
    if english(name) == True:
        english_ios_apps.append(app)

english_android_apps = []
for app in android_clean:
    name = app[0]
    if english(name) == True:
        english_android_apps.append(app)   
        
explore_data(english_ios_apps, 1, 10)
explore_data(english_android_apps, 1, 10)
print('\n')
print('Number of English iOS Apps:', len(english_ios_apps))
print('\n')
print('Number of English Android Apps:', len(english_android_apps))

The final cleaning step is to isolate the free apps in a separate list.

In [ ]:
final_ios = []
for app in english_ios_apps:
    price = app[4]
    if float(price) == 0.0:
        final_ios.append(app)
    
final_android = []
for app in english_android_apps:
    price = app[7]
    if price == '0':
        final_android.append(app)    

explore_data(final_ios, 1, 10)
explore_data(final_android, 1, 10)

print(len(final_ios))
print(len(final_android))
        

My validation strategy for an app idea is as follows:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, develop it further.
3. If the app is profitable after six months, build an iOS version of the app and add it to the App Store.

To analyze the app data, I will build frequency tables. To do this, I will build one function to generate frequency tables that show percentages, and another function to display percentages in a descending order using the `sorted()` function. To do this, I will have to transform the dictionary into a list of tuples, with each one containing a dictionary key and its corresponding value. The value must come first in order to sort. 

The `display_table()` function below takes in two parameters, `dataset` (a list of lists) and `index` (an integer). It generates a frequency table using the function `freq_table()`, transforms tje frequency table into a list of tuples, sorts the list in descending order, and prints the table.  

In [ ]:
def freq_table(dataset, index):
    table = {}
    for app in dataset:
        genre = app[index]
        if genre in table:
            table[genre] += 1
        else:
            table[genre] = 1
    for genre in table:
        table[genre] = (table[genre] / len(dataset)) * 100
    return table        

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
    table_sorted = sorted(table_display, reverse=True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

I will start by examining the frequency table for the iOS apps using the `prime_genre` column. 

In [ ]:
display_table(final_ios, 11)

Among the free English apps on iOS, the highest frequency belongs to the category "Games", which is 58.16% of apps. Apps in the category "Entertainment" have a much lower frequency of 7.88%. The third most frequent category is "Photo & Video" at 4.97%. 

I will now analyze the Android apps, first using the Category column.

In [ ]:
display_table(final_android, 1)

The frequencies are quite different for the Android apps. The most frequent category is "Family", which has a percentage of 18.91%, while the next highest category, "Games", falls at just 9.72%. About 10% for Android compared to almost 60% for iOS for games is very interesting. To investigate further, we can see that the "Family" category actually does have several games, mainly for children. Therefore, the actual "Game" frequency may be closer to the sum of the highest 2 categories, about 30%. However, we can determine that the App Store seems to be dominated by gaming and entertainment apps, whereas the Google Play store tends to have a more balanced selection.

Additionally, there are more Android categories than iOS genres, which may be why the frequencies are more dispersed.

Next, I will analyze the Android apps using the Genre column.

In [ ]:
display_table(final_android, 9) 

We can see that there are many more options for Genre on the Android data, so we will be using the Category data moving forward. 

The information we have at the moment does not provide enough to recommend an app profile, and the frequency tables do not necessarily reveal which genres have the most users. 

In order to determine which apps have the most users, I will analyze the number of `Installs` on the Android apps. This information is not available for the iOS apps, so I will be using the total number of user ratings as a proxy (column `rating_count_tot`). 

In [ ]:
freq_ios = freq_table(final_ios, 11)
unique_genres = []
for row in freq_ios:
    unique_genres.append(row)
print(unique_genres)

In [ ]:
for genre in unique_genres:
    total = 0   # store the sum of user ratings 
    len_genre = 0   # number of apps specific to each genre
    for row in final_ios:
        genre_app = row[11]
        if str(genre_app) == str(genre):
            user_ratings = int(row[5])
            total += user_ratings
            len_genre += 1
            
    avg_num_rat = total / len_genre
    print(genre, ':', avg_num_rat)

We can see that apps in the "Navigation" genre have the  greatest number of user ratings. Next to "Navigation" is the "Reference" genre, and "Social Networking" comes in third place. For free, English apps on iOS, I recommend a Social Networking genre. 

In [ ]:
display_table(final_android, 5)

We can see above that the `Installs` column of the Android table does not show precise numbers, but rather open ended ones. I will consider an app with 100,000+ installs as having 100,000. To do so, I will have to convert each of these strings to a float, meaning I have to remove the commas and the plus character. Below is an example of how I will do this.

In [ ]:
n_installs = '100,000+'
n_installs = n_installs.replace('+', '')
n_installs = n_installs.replace(',', '')
print(n_installs)

Now, I will calculate the average number of installs per app genre for the Google Play data set. 

In [ ]:
freq_android = freq_table(final_android, 1)
unique_categories = []
for row in freq_android:
    unique_categories.append(row)
print(unique_categories)

In [ ]:
for category in unique_categories:
    total = 0   # store the sum of installs per category
    len_category = 0   # number of apps specific to each category
    for row in final_android:
        category_app = row[1]
        if str(category_app) == str(category):
            user_installs = row[5]
            user_installs = user_installs.replace('+', '')
            user_installs = user_installs.replace(',', '')
            user_installs = int(user_installs)
            total += user_installs
            len_category += 1
            
    avg_num_cat = total / len_category
    print(category, ':', avg_num_cat)

It looks as though the "Communication" category has the greatest number of installs, with "Social" being next. The app profile I recommend is "Communication".

To futher analyze this data, I can: 
* Look into the frequence table for the `Genre` column of the Google Play data set for patterns. 
* Try to determine which genres seem to be most liked by users by examining app ratings.
* Refine this project using a data science project [style guide](https://www.dataquest.io/blog/data-science-project-style-guide/)
* Use [Stack Overflow](https://stackoverflow.com/) for "python" questions.